<a href="https://colab.research.google.com/github/afrojaakter/stylegan2/blob/master/Art2000Stylegan2FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [ ]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle/abstract-art-gallery

In [ ]:
#downloading dataset from kaggle
!kaggle datasets download -d bryanb/abstract-art-gallery

In [ ]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

In [ ]:
#run it only once.
#creating repositories for stylegan2
%cd /content/gdrive/My\ Drive/
!mkdir stylegan2-colab-abstract-art-gallery
%cd stylegan2-colab-abstract-art-gallery/
!git clone https://github.com/dvschultz/stylegan2
%cd stylegan2
!mkdir pkl
%cd pkl
!gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF 
%cd ../
!mkdir results
!mkdir results/00001-pretrained
%cd results/00001-pretrained
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
!mv stylegan2-ffhq-config-f.pkl network-snapshot-10000.pkl
%cd ../../
%mkdir datasets
%mkdir datasets/tfRecords
%mkdir datasets/SquareImages

In [ ]:
%tensorflow_version 1.x
#USE this if you already have a stylegan2 folder in google drive
%cd /content/gdrive/My\ Drive/stylegan2-colab-abstract-art-gallery/stylegan2
!git pull

In [ ]:
#!/usr/bin/python
from PIL import Image
import os, sys

path = "/content/gdrive/MyDrive/Kaggle/abstract-art-gallery/Abstract_gallery/" 
#path2 = "/content/gdrive/MyDrive/Kaggle/images/images/William_Turner/"
#path = os.path.join(path1, path2)
dirs = os.listdir(path)

def resize_dataset():
    for item in dirs:
        #print(item)
        im = Image.open(path+item)
        f, e = os.path.splitext(path+item)
        imResize = im.resize((1024,1024), Image.ANTIALIAS)
        imResize.save('/content/gdrive/MyDrive/stylegan2-colab-abstract-art-gallery/stylegan2/datasets/SquareImages/' + item, 'JPEG', quality=90)

resize_dataset()

In [ ]:
#2nd argument is where to put your tfrecords files
#3rd should point at your image dataset
!python dataset_tool.py create_from_images ./Datasets/tfRecords /content/gdrive/MyDrive/stylegan2-colab-abstract-art-gallery/stylegan2/datasets/SquareImages

In [ ]:
!python run_training.py --num-gpus=1 --data-dir=./Datasets --config=config-f --dataset=tfRecords --mirror-augment=true --metrics=fid50k

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/

In [ ]:
!pip install opensimplex

In [ ]:
!python run_generator.py generate-images --network=/content/gdrive/MyDrive/stylegan2-colab-abstract-art-gallery/stylegan2/results/00017-stylegan2-tfRecords-1gpu-config-f/network-snapshot-010851.pkl --seeds=1-60 --truncation-psi=1.0

In [ ]:
!zip -r generated-0.4.zip /content/gdrive/MyDrive/stylegan2-colab/stylegan2/results/00010-generate-images

In [ ]:
!python run_generator.py generate-latent-walk --network=/content/gdrive/MyDrive/stylegan2-colab-abstract-art-gallery/stylegan2/results/00017-stylegan2-tfRecords-1gpu-config-f/network-snapshot-010851.pkl --seeds=3,11,17,25,3 --frames 200 --truncation-psi=0.7

In [ ]:
#convert to video /content/gdrive/MyDrive/stylegan2-colab-abstract-art-gallery/stylegan2/results/00005-generate-latent-walk
!ffmpeg -r 24 -i ./results/00005-generate-latent-walk/frame%05d.png -vcodec libx264 -pix_fmt yuv420p latent-walk-v2.mp4

In [ ]:
ls

In [ ]:
!ffmpeg -r 24 -i ./results/00008-generate-images/seed%4d.png -vcodec libx264 -pix_fmt yuv420p latent-walk-v2.mp4

In [ ]:
#/content/gdrive/MyDrive/stylegan2-colab-abstract-art-gallery/stylegan2/results/00007-stylegan2-tfRecords-1gpu-config-f
!ffmpeg -r 24 -i ./results/00007-stylegan2-tfRecords-1gpu-config-f/fakes01%5d.jpg -vcodec libx264 -pix_fmt yuv420p testwalk-v2.mp4

In [ ]:
#/content/gdrive/MyDrive/stylegan2-colab-abstract-art-gallery/stylegan2/results/00007-stylegan2-tfRecords-1gpu-config-f

#%cd /content/gdrive/My\ Drive/stylegan2-colab-abstract-art-gallery/stylegan2/results/00007-stylegan2-tfRecords-1gpu-config-f
!ffmpeg -pattern_type glob -i '*.jpg' -vcodec libx264 out.mp4

In [ ]:
#!ffmpeg -r 60 -f image2 -s 1920x1080 -start_number 010000 -i fakes%06d.jpg -vframes 100 -vcodec libx264 -crf 25  -pix_fmt yuv420p test1.mp4
!ffmpeg -pattern_type glob -i '*.jpg' -vcodec libx264 out.mp4

In [ ]:
!ffmpeg -start_number 010000 -i 'fakes0%5d.jpg' -c:v libx264 out.mp4